In [16]:
#this python script is collecting different instance raw data and generate LT reports
import numpy as np
import pandas as pd
import cufflinks as cf
import matplotlib.pyplot as plt
import seaborn as sns
import os
import datetime

#set config file for cufflinks library
cf.set_config_file(offline=True)

#import as dataframe the raw data in each instance and store into a list of dataframes called df
def upload_data(ST_instances):
    """upload each instance raw data file found in ST_instances passed list into a dataframe 
        and returtn df as a lits of dataframes"""      
    df=[]
    for i in range(len(ST_instances)):
        print(f"uploaded raw date from instance_{i}")
        filename=ST_instances[i]
        print(filename)
        filepath=os.path.join(working_dir,filename)
        dfi=pd.read_excel(filepath)
        df.append(dfi)
    return df

#generate the merged instance datasframe called data and store into an excel file
def merge_data(data_sources):
    """merge all uploaded dataframes into a single master dataframe called df and save it to 
    an excel file called merged data with datetime tag as extension"""
    data=pd.DataFrame(data_sources[0])  #generate from first dataframe
    ext=datetime.datetime.now().strftime("%Y_%m_%d")
    merged_filename=f"merged_data_{ext}"
    for i in range(1,len(data_sources)):
        data=pd.concat([data,data_sources[i]],axis=0)
    df=data.copy().reset_index().drop(columns="index")
    df.to_excel(f"{merged_filename}.xlsx")
    return df

def generate_pivot(df,MA,key):
    """generate a pivot a pivot per MA with index=key where key can be either CU or type of project,
       while the aggregation funtion calculated are the mean and the standard deviations.
       the function return the df_pivot dataframe and save also this inbto an excel file
    """
    df_MA=df[df["MA"]==MA]
    df_pivot=df_MA.pivot_table(index=key,aggfunc=["mean","std"])
    #df_pivot.loc[:,("mean")].plot(figsize=(16,9),kind="barh",title=f"{MA} LT mean {key} performances")
    #df_pivot.loc[:,("std")].plot(figsize=(16,9),kind="barh",title=f"{MA} LT std {key} performances")
    filename=f"{MA}_{key}.xlsx"
    df_pivot.to_excel(filename)
    return np.round(df_pivot,2)

def plot_perf(df_pivot,MA,aggfunc="mean"):
    #plt.figure(figsize=(16,9))
    df_pivot.loc[:,(aggfunc)].iplot(kind="barh",title=f"{MA} LTs measurement = {aggfunc}")    
    
def plot_MA_statistics(df,MA,LT_columns):
    return df[df["MA"]==MA][LT_columns].iplot(kind="box",x="CU",title=f"{MA} LT statistics",yTitle="calendar days")


#set working dir path
working_dir=input("Provide working dir path:\n").replace("'\'","/")
os.chdir(working_dir)

#select the list of ST instances raw files in working_dir names to source  data from
ST_instances=[]
file_list=os.listdir()
for file in file_list:
    if file.find("instance")!= -1:
        #print(file)
        ST_instances.append(file)

#upload raw data
data_sources=upload_data(ST_instances)

#call function to merge data into a master dataframe df
df=merge_data(data_sources)
LT_columns=df.columns[-10:]

#set the MA names in a list called MA_list
MAs=df.groupby("MA")["MA"].value_counts().index
MA_list=[]
for i in range(len(MAs)):
    MA_list.append(MAs[i][0])
    
LT_statistics=df[df["MA"]=="MA1"][LT_columns].describe()
LT_statistics.to_excel("LT_statistics.xlsx")
print(np.round(LT_statistics,2))

print("********************* plotting LT performances ******************************************")

pivot_key=input("Provide pivot_key: [CU,project_type]: ")

for MA in MA_list:
    MA_perf=generate_pivot(df,MA,pivot_key)
    plot_perf(MA_perf,MA)
    plot_MA_statistics(df,MA,LT_columns)


Provide working dir path:
C:\Users\EPARDET\OneDrive - Ericsson\area lavoro\Nello\Governance meeting\switch program 2022\LT investigations\2023 LT OKR\2023 execution\python script
uploaded raw date from instance_0
ST_instance1.xlsx
uploaded raw date from instance_1
ST_instance2.xlsx
          LT1     LT2     LT3     LT4     LT5     LT6     LT7     LT8     LT9  \
count  125.00  125.00  125.00  125.00  125.00  125.00  125.00  125.00  125.00   
mean    74.68   20.62   11.33    9.53   14.46    2.51    4.50   20.74   45.66   
std     10.31    3.04    1.75    1.34    2.34    0.53    0.81    3.85    6.44   
min     48.00   12.00    7.00    6.00    7.00    2.00    2.00   12.00   31.00   
25%     68.00   19.00   10.00    9.00   13.00    2.00    4.00   18.00   42.00   
50%     74.00   21.00   11.00   10.00   14.00    2.00    4.00   20.00   46.00   
75%     82.00   23.00   13.00   10.00   16.00    3.00    5.00   23.00   50.00   
max    102.00   27.00   18.00   13.00   20.00    4.00    6.00   33.00